<a href="https://colab.research.google.com/github/DhawaDong/Breast-Cancer-ANN/blob/main/ANN_BreastCancerDiagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd;
from sklearn.model_selection import train_test_split;
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
DataSetBC = pd.read_csv(r'/content/drive/MyDrive/BreastCancer#MachineLearning/BreastCancerDataSet.csv')

In [ ]:
DataSetBC.shape

(569, 33)

In [ ]:
x = DataSetBC.drop(['id', 'diagnosis', 'Unnamed: 32'], axis =1);
y = DataSetBC['diagnosis'];

In [ ]:
DiagClass = {'M': 1.0, 'B': 0.0}
y = y.map(DiagClass);

In [ ]:
x.shape, y.shape

((569, 30), (569,))

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

#y_test_array = np.array(y_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# Define the ANN model
class BreastCancerANN(nn.Module):
    def __init__(self):
        super(BreastCancerANN, self).__init__()
        self.layer1 = nn.Linear(30, 16)  # 30 input features, 16 hidden units
        self.layer2 = nn.Linear(16, 1)   # 16 hidden units, 1 output unit
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
# Instantiate the model
model = BreastCancerANN()

In [ ]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.view(-1, 1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/1000], Loss: 0.5955
Epoch [20/1000], Loss: 0.5185
Epoch [30/1000], Loss: 0.4525
Epoch [40/1000], Loss: 0.3966
Epoch [50/1000], Loss: 0.3486
Epoch [60/1000], Loss: 0.3073
Epoch [70/1000], Loss: 0.2723
Epoch [80/1000], Loss: 0.2430
Epoch [90/1000], Loss: 0.2186
Epoch [100/1000], Loss: 0.1983
Epoch [110/1000], Loss: 0.1814
Epoch [120/1000], Loss: 0.1671
Epoch [130/1000], Loss: 0.1550
Epoch [140/1000], Loss: 0.1447
Epoch [150/1000], Loss: 0.1357
Epoch [160/1000], Loss: 0.1278
Epoch [170/1000], Loss: 0.1209
Epoch [180/1000], Loss: 0.1148
Epoch [190/1000], Loss: 0.1093
Epoch [200/1000], Loss: 0.1044
Epoch [210/1000], Loss: 0.0999
Epoch [220/1000], Loss: 0.0958
Epoch [230/1000], Loss: 0.0922
Epoch [240/1000], Loss: 0.0889
Epoch [250/1000], Loss: 0.0858
Epoch [260/1000], Loss: 0.0831
Epoch [270/1000], Loss: 0.0805
Epoch [280/1000], Loss: 0.0782
Epoch [290/1000], Loss: 0.0760
Epoch [300/1000], Loss: 0.0741
Epoch [310/1000], Loss: 0.0722
Epoch [320/1000], Loss: 0.0705
Epoch [330/1000],

In [ ]:
# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    predictions = model(X_test_tensor)
    predictions = (predictions >= 0.5).float()  # Convert probabilities to binary predictions
    accuracy = torch.sum(predictions == y_test_tensor.view(-1, 1)).item() / y_test_tensor.size(0)

In [ ]:
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.9912
